#**Практическое задание 5**

## Часть 1

In [9]:
%%writefile stack_cuda.cu

// stack_cuda.cu
// Часть 1: Параллельный стек на CUDA с атомарными операциями (push/pop)

#include <iostream>                 // cout для вывода
#include <vector>                   // vector для удобных массивов на CPU
#include <cuda_runtime.h>           // CUDA runtime API (cudaMalloc/cudaMemcpy/events)

using namespace std;                // чтобы не писать std:: каждый раз

// Макрос: проверка ошибок CUDA-вызовов (если ошибка - печатаем и завершаем программу)
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)

// Структура стека на GPU

// Стек хранится в глобальной памяти GPU, а top меняется атомарно
struct Stack {                                                      // объявляем структуру Stack
    int *data;                                                      // указатель на массив данных в памяти GPU
    int top;                                                        // индекс вершины стека (последний занятый элемент)
    int capacity;                                                   // максимальный размер стека

    __device__ void init(int *buffer, int size) {                   // device-функция инициализации стека на GPU
        data = buffer;                                              // запоминаем адрес массива данных
        top = -1;                                                   // стек пустой, поэтому top = -1
        capacity = size;                                            // сохраняем емкость
    }

    __device__ bool push(int value) {                               // device-функция push: кладем значение в стек
        int pos = atomicAdd(&top, 1) + 1;                           // атомарно увеличиваем top и получаем новую позицию
        if (pos < capacity) {                                       // проверяем, что стек не переполнен
            data[pos] = value;                                      // записываем значение в стек
            return true;                                            // push успешен
        } else {                                                    // если переполнение
            atomicSub(&top, 1);                                     // откатываем top назад, чтобы не ломать состояние
            return false;                                           // push не выполнен
        }
    }

    __device__ bool pop(int *value) {                               // device-функция pop: достаем значение из стека
        int pos = atomicSub(&top, 1);                               // атомарно уменьшаем top, pos - старое значение top
        if (pos >= 0) {                                             // если было что доставать (pos был валидным индексом)
            *value = data[pos];                                     // забираем значение из data[pos]
            return true;                                            // pop успешен
        } else {                                                    // если стек был пуст
            atomicAdd(&top, 1);                                     // возвращаем top обратно (откат)
            return false;                                           // pop не выполнен
        }
    }
};

// CUDA kernel: параллельные push/pop

// Каждый поток: делает push своего id, потом часть потоков делает pop
__global__ void testStackKernel(Stack *st, int *popOut, int N) {     // kernel, st - стек, popOut - массив результатов pop, N - число потоков
    int tid = blockIdx.x * blockDim.x + threadIdx.x;                // считаем глобальный id потока
    if (tid >= N) return;                                           // защита, если потоков запустили больше чем N

    bool okPush = st->push(tid);                                    // пытаемся положить tid в стек

    __syncthreads();                                                // синхронизация внутри блока (чтобы чуть стабилизировать картину)

    int v = -1;                                                     // сюда будем доставать значение
    bool okPop = false;                                             // флаг успешности pop

    if (okPush) {                                                   // pop имеет смысл только если push был успешен
        okPop = st->pop(&v);                                        // пытаемся сделать pop
    }

    // Записываем результат: если pop успешен, пишем значение, иначе -1
    popOut[tid] = (okPop ? v : -1);                                 // сохраняем результат для проверки на CPU
}

// Инициализация стека на GPU

// kernel для init, потому что init - device-функция (ее нельзя вызвать напрямую с CPU)

__global__ void initStackKernel(Stack *st, int *buffer, int cap) {   // kernel инициализации стека
    if (threadIdx.x == 0 && blockIdx.x == 0) {                      // только один поток реально делает init
        st->init(buffer, cap);                                      // вызываем init на устройстве
    }
}

// main

int main() {                                                        // старт программы
    const int N = 1024;                                             // сколько потоков будем тестировать (push/pop)
    const int CAP = 1024;                                           // емкость стека (фиксированная по заданию)

    // Выделение памяти на GPU

    Stack *dStack = nullptr;                                        // указатель на Stack в GPU памяти
    int *dBuffer = nullptr;                                         // буфер данных стека в GPU памяти
    int *dPopOut = nullptr;                                         // массив результатов pop в GPU памяти

    CUDA_CHECK(cudaMalloc(&dStack, sizeof(Stack)));                 // выделяем память под структуру Stack
    CUDA_CHECK(cudaMalloc(&dBuffer, CAP * sizeof(int)));            // выделяем память под данные стека
    CUDA_CHECK(cudaMalloc(&dPopOut, N * sizeof(int)));              // выделяем память под массив результатов pop

    // Инициализация стека

    initStackKernel<<<1, 1>>>(dStack, dBuffer, CAP);                // запускаем kernel init (1 блок, 1 поток)
    CUDA_CHECK(cudaGetLastError());                                 // проверяем запуск
    CUDA_CHECK(cudaDeviceSynchronize());                            // ждём завершения init

    // Запуск тестового kernel

    dim3 block(256);                                                // выбрали размер блока
    dim3 grid((N + block.x - 1) / block.x);                         // считаем grid.x, чтобы покрыть N потоков

    testStackKernel<<<grid, block>>>(dStack, dPopOut, N);           // запускаем kernel с push/pop
    CUDA_CHECK(cudaGetLastError());                                 // проверяем запуск
    CUDA_CHECK(cudaDeviceSynchronize());                            // ждём завершения вычислений

    // Копируем результаты pop на CPU

    vector<int> hPop(N);                                            // массив на CPU для результатов pop
    CUDA_CHECK(cudaMemcpy(hPop.data(), dPopOut, N * sizeof(int), cudaMemcpyDeviceToHost)); // копируем из GPU в CPU

    // Проверка корректности

    // Все значения должны быть из диапазона [0..N-1] или -1,
    // и одинаковых значений (кроме -1) быть не должно, иначе стек отдал дубликаты

    vector<int> seen(N, 0);                                         // seen[i]=1 если значение i уже встретили
    int ok = 1;                                                     // флаг корректности
    int poppedCount = 0;                                            // сколько успешных pop

    for (int i = 0; i < N; ++i) {                                   // пробегаем по всем потокам
        int v = hPop[i];                                            // берем то, что поток достал из стека
        if (v == -1) continue;                                      // -1 значит pop не получился (например overflow/empty)
        if (v < 0 || v >= N) {                                      // если достали что-то вне диапазона
            ok = 0;                                                 // это ошибка
            break;                                                  // дальше можно не проверять
        }
        if (seen[v]) {                                              // если значение уже встречали
            ok = 0;                                                 // значит дубликат, это ошибка
            break;                                                  // выходим
        }
        seen[v] = 1;                                                // отмечаем, что значение v уже было
        poppedCount++;                                              // увеличиваем счетчик успешных pop
    }

    // Печать результата

    cout << "Parallel stack test\n";
    cout << "Threads N = " << N << "\n";                             // сколько потоков
    cout << "Stack capacity = " << CAP << "\n";                      // емкость стека
    cout << "Popped values (non -1) = " << poppedCount << "\n";      // сколько реально достали
    cout << "Correctness: " << (ok ? "OK" : "FAIL") << "\n";         // итог проверки

    // Освобождение памяти

    CUDA_CHECK(cudaFree(dStack));                                   // освобождаем Stack
    CUDA_CHECK(cudaFree(dBuffer));                                  // освобождаем буфер данных
    CUDA_CHECK(cudaFree(dPopOut));                                  // освобождаем результаты

    return 0;                                                       // конец программы
}


Overwriting stack_cuda.cu


In [10]:
!nvcc -O3 -arch=sm_75 stack_cuda.cu -o stack_cuda
!./stack_cuda

Parallel stack test
Threads N = 1024
Stack capacity = 1024
Popped values (non -1) = 1024
Correctness: OK


##**Вывод**
В ходе выполнения работы была реализована параллельная структура данных стек на GPU с использованием атомарных операций для безопасного доступа к данным из множества потоков. Стек был инициализирован с фиксированной емкостью, после чего выполнялись параллельные операции push и pop в CUDA-ядре.

Результаты эксперимента показали, что все 1024 потока смогли корректно записать и затем извлечь значения из стека. Количество успешно извлеченных элементов совпало с количеством выполненных операций push, что подтверждает отсутствие потерь данных и корректность работы алгоритма. Использование атомарных операций atomicAdd и atomicSub обеспечило синхронизацию потоков и предотвратило состояния гонки при одновременном доступе к переменной top.

Таким образом, реализованный параллельный стек корректно функционирует в условиях многопоточного доступа и может быть использован как базовый механизм синхронизированного хранения данных на GPU.

## Часть 2

In [16]:
%%writefile queue_vs_stack.cu
#include <iostream>              // cout
#include <vector>                // vector на CPU
#include <cuda_runtime.h>        // CUDA runtime + events
#include <iomanip>               // setw, setprecision
#include <cmath>                 // fabs

using namespace std;

// Проверка CUDA ошибок: если что-то пошло не так, сразу печатаем и выходим
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)

// Stack (из Part 1, чтобы сравнить)
struct Stack {
  int *data;        // буфер элементов в global memory
  int top;          // индекс вершины
  int capacity;     // емкость

  __device__ void init(int *buffer, int size) {
    data = buffer;          // привязываем буфер
    top = -1;               // пустой стек
    capacity = size;        // сохраняем емкость
  }

  __device__ bool push(int value) {
    int pos = atomicAdd(&top, 1) + 1;   // резервируем позицию (top увеличили), pos = новый top
    if (pos < capacity) {              // если не вышли за границы
      data[pos] = value;               // кладем элемент
      return true;                     // успех
    }
    atomicSub(&top, 1);                // откат, если переполнили
    return false;                      // не удалось
  }

  __device__ bool pop(int *value) {
    int pos = atomicSub(&top, 1);      // забираем текущий top и уменьшаем
    if (pos >= 0) {                    // если стек был не пустой
      *value = data[pos];              // читаем значение
      return true;                     // успех
    }
    atomicAdd(&top, 1);                // откат, если стек был пустой
    return false;                      // не удалось
  }
};

// Queue (Part 2)
struct Queue {
  int *data;        // буфер элементов
  int head;         // откуда читаем
  int tail;         // куда пишем
  int capacity;     // емкость

  __device__ void init(int *buffer, int size) {
    data = buffer;          // привязка буфера
    head = 0;               // начало очереди
    tail = 0;               // конец очереди
    capacity = size;        // емкость
  }

  __device__ bool enqueue(int value) {
    int pos = atomicAdd(&tail, 1);     // резервируем позицию в конце
    if (pos < capacity) {              // если не вышли за емкость
      data[pos] = value;               // пишем значение
      return true;                     // успех
    }

    return false;                      // не удалось
  }

  __device__ bool dequeue(int *value) {
    int pos = atomicAdd(&head, 1);     // резервируем позицию для чтения
    // Сравниваем с tail (сколько реально добавили)
    // Здесь для корректности мы будем вызывать dequeue только после того, как все enqueue закончились в блоке
    if (pos < tail) {                  // если в очереди есть элементы
      *value = data[pos];              // читаем элемент
      return true;                     // успех
    }
    return false;                      // пусто
  }
};

// Kernels

// Инициализация стека (один поток)

__global__ void initStackKernel(Stack *S, int *buf, int cap) {
  if (threadIdx.x == 0) S->init(buf, cap);
}

// Инициализация очереди (один поток)

__global__ void initQueueKernel(Queue *Q, int *buf, int cap) {
  if (threadIdx.x == 0) Q->init(buf, cap);
}

// Тест стека: все потоки делают push, потом pop (внутри одного блока)

__global__ void stackTestKernel(Stack *S, int *out, int nThreads) {
  int tid = threadIdx.x;

  if (tid < nThreads) {
    S->push(tid);                 // каждый поток кладет свой tid
  }
  __syncthreads();                // ждём пока все push закончатся

  int val = -1;                   // если pop не удастся, останется -1
  if (tid < nThreads) {
    S->pop(&val);                 // каждый поток пытается вытащить
    out[tid] = val;               // записываем результат
  }
}

// Тест очереди: сначала enqueue от всех потоков, потом dequeue от всех потоков (внутри одного блока)

__global__ void queueTestKernel(Queue *Q, int *out, int nThreads) {
  int tid = threadIdx.x;

  if (tid < nThreads) {
    Q->enqueue(tid);              // каждый поток добавляет свой tid в хвост
  }
  __syncthreads();                // после этого tail уже “зафиксирован” для блока

  int val = -1;                   // если dequeue не удастся, будет -1
  if (tid < nThreads) {
    Q->dequeue(&val);             // каждый поток забирает один элемент
    out[tid] = val;               // пишем что забрали
  }
}


// Замеряем только kernel time через CUDA events

template <typename Launch>
float timeKernel(Launch launch) {
  cudaEvent_t e1, e2;
  CUDA_CHECK(cudaEventCreate(&e1));
  CUDA_CHECK(cudaEventCreate(&e2));

  CUDA_CHECK(cudaEventRecord(e1));
  launch();
  CUDA_CHECK(cudaEventRecord(e2));
  CUDA_CHECK(cudaEventSynchronize(e2));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, e1, e2));

  CUDA_CHECK(cudaEventDestroy(e1));
  CUDA_CHECK(cudaEventDestroy(e2));
  return ms;
}


// Проверка очереди: должны получить ровно nThreads значений из диапазона [0..nThreads-1], без повторов

bool checkQueueResult(const vector<int>& out, int nThreads) {
  vector<int> seen(nThreads, 0);

  for (int i = 0; i < nThreads; i++) {
    int v = out[i];
    if (v < 0 || v >= nThreads) return false;  // вылезли за диапазон или -1
    if (seen[v]) return false;                 // повтор
    seen[v] = 1;
  }
  return true;
}

// Проверка стека: тоже должны получить nThreads валидных значений без -1
// (порядок для стека будет LIFO, но нам важно что все элементы реально достались)

bool checkStackResult(const vector<int>& out, int nThreads) {
  vector<int> seen(nThreads, 0);

  for (int i = 0; i < nThreads; i++) {
    int v = out[i];
    if (v < 0 || v >= nThreads) return false;
    if (seen[v]) return false;
    seen[v] = 1;
  }
  return true;
}

int main() {
  const int NTHREADS = 1024;                 // как у нас в стеке, максимум для одного блока
  const int CAPACITY = 1024;                // емкость очереди/стека

  // Выделяем память на GPU под буферы данных
  int *dBufStack = nullptr, *dBufQueue = nullptr;
  int *dOutStack = nullptr, *dOutQueue = nullptr;

  CUDA_CHECK(cudaMalloc(&dBufStack, CAPACITY * sizeof(int)));
  CUDA_CHECK(cudaMalloc(&dBufQueue, CAPACITY * sizeof(int)));
  CUDA_CHECK(cudaMalloc(&dOutStack, NTHREADS * sizeof(int)));
  CUDA_CHECK(cudaMalloc(&dOutQueue, NTHREADS * sizeof(int)));

  // Выделяем память на GPU под сами структуры Stack и Queue
  Stack *dStack = nullptr;
  Queue *dQueue = nullptr;
  CUDA_CHECK(cudaMalloc(&dStack, sizeof(Stack)));
  CUDA_CHECK(cudaMalloc(&dQueue, sizeof(Queue)));

  // Инициализируем структуры на GPU
  initStackKernel<<<1, 1>>>(dStack, dBufStack, CAPACITY);
  initQueueKernel<<<1, 1>>>(dQueue, dBufQueue, CAPACITY);
  CUDA_CHECK(cudaDeviceSynchronize());

  // Настройки запуска: один блок и NTHREADS потоков
  dim3 block(NTHREADS);
  dim3 grid(1);

  // Прогрев, чтобы первый запуск не портил измерение
  stackTestKernel<<<grid, block>>>(dStack, dOutStack, NTHREADS);
  queueTestKernel<<<grid, block>>>(dQueue, dOutQueue, NTHREADS);
  CUDA_CHECK(cudaDeviceSynchronize());

  // Замер времени стека
  float tStack = timeKernel([&] {
    initStackKernel<<<1, 1>>>(dStack, dBufStack, CAPACITY);          // заново обнуляем top
    stackTestKernel<<<grid, block>>>(dStack, dOutStack, NTHREADS);
  });

  // Замер времени очереди
  float tQueue = timeKernel([&] {
    initQueueKernel<<<1, 1>>>(dQueue, dBufQueue, CAPACITY);          // заново обнуляем head/tail
    queueTestKernel<<<grid, block>>>(dQueue, dOutQueue, NTHREADS);
  });

  // Скачиваем результаты на CPU для проверки корректности
  vector<int> hStack(NTHREADS), hQueue(NTHREADS);
  CUDA_CHECK(cudaMemcpy(hStack.data(), dOutStack, NTHREADS * sizeof(int), cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hQueue.data(), dOutQueue, NTHREADS * sizeof(int), cudaMemcpyDeviceToHost));

  bool okStack = checkStackResult(hStack, NTHREADS);
  bool okQueue = checkQueueResult(hQueue, NTHREADS);

  // Красивый вывод
  cout << "Parallel data structures test (CUDA)\n";
  cout << "Threads N = " << NTHREADS << "\n";
  cout << "Capacity  = " << CAPACITY << "\n\n";

  cout << left << setw(12) << "Structure"
       << setw(18) << "Kernel time (ms)"
       << setw(14) << "Correctness"
       << "\n";

  cout << left << setw(12) << "Stack"
       << setw(18) << fixed << setprecision(6) << tStack
       << setw(14) << (okStack ? "OK" : "FAIL")
       << "\n";

  cout << left << setw(12) << "Queue"
       << setw(18) << fixed << setprecision(6) << tQueue
       << setw(14) << (okQueue ? "OK" : "FAIL")
       << "\n";

  // Освобождение GPU памяти
  CUDA_CHECK(cudaFree(dBufStack));
  CUDA_CHECK(cudaFree(dBufQueue));
  CUDA_CHECK(cudaFree(dOutStack));
  CUDA_CHECK(cudaFree(dOutQueue));
  CUDA_CHECK(cudaFree(dStack));
  CUDA_CHECK(cudaFree(dQueue));

  return 0;
}


Overwriting queue_vs_stack.cu


In [18]:
!nvcc -O3 -arch=sm_75 queue_vs_stack.cu -o queue_vs_stack
!./queue_vs_stack

Parallel data structures test (CUDA)
Threads N = 1024
Capacity  = 1024

Structure   Kernel time (ms)  Correctness   
Stack       0.013824          OK            
Queue       0.010368          OK            


## **Вывод**
В ходе выполнения задания была реализована параллельная структура данных стек и очередь на CUDA с использованием атомарных операций для безопасного доступа из нескольких потоков. Тестирование проводилось для 1024 потоков при фиксированной ёмкости 1024 элементов, при этом измерялось только время выполнения ядра без учёта передачи данных между CPU и GPU.

Обе структуры корректно обработали все операции push/pop и enqueue/dequeue, что подтверждается результатом проверки корректности. По времени выполнения очередь показала более высокую производительность (0.010368 мс) по сравнению со стеком (0.013824 мс).

Разница в производительности связана с особенностями работы атомарных операций. В стеке все потоки конкурируют за одну переменную top, что увеличивает задержки при параллельном доступе. В очереди операции распределены между двумя переменными head и tail, что снижает уровень конфликтов между потоками и делает работу более эффективной.

Таким образом, при высокой степени параллелизма очередь может работать быстрее, чем стек, благодаря меньшей нагрузке на атомарные операции.